# Selecting Data from PostgreSQL

## Introduction

Now that you've gotten a brief introduction to SQL, its time to get some hands on practice connecting to a database via Python and executing to some queries.

## Objectives

You will be able to:

* Understand the basic structure of a `SELECT` statement in SQL

## Connecting To a Database

First let's connect to our database by importing psycopg2 and running the following cell in our notebook. You'll need a cursor object (`cur`) to fetch results. Cursor objects allow you to keep track of which result set is which, since it's possible to run multiple queries before you're done fetching the results of the first.

```python 
import psycopg2
conn = psycopg2.connect('data_base_infos')
cur = conn.cursor()
```
you will not have `psycog2` already installed in your anaconda enviroment, so go ahead and execute the following to install pyscog2 to your `(base)(Python3.x.x)`

```python
$ conda activate base 
$ pip install psycopg2
$ conda deactivate
$ conda list  ** to check
```

In [1]:
# import postgresql realated libs 
import psycopg2
import pandas as pd 
import run_utility as ru 
import pyodbc

In [19]:
# ConfigParser parses the config file 
from configparser import ConfigParser
config = ConfigParser()
config.read('config.ini')

['config.ini']

In [20]:
print(config.sections())

['dev', 'client', 'dvdrental']


Get the connection credential to connect to postgresql

In [5]:
# variable = config[section][what_we_want]
database = config['dvdrental']['database']
host = config['dvdrental']['host']
port = config['dvdrental']['port']
user = config['dvdrental']['user']
password = config['dvdrental']['password']

print('Database: ', database)
print('Host: ', host)
print('Port: ', port)
print('User: ', user)

Database:  dev
Host:  localhost
Port:  5432
User:  postgres


In [6]:
# creating a connection to postgresql
conn= ru.create_connection(
    databasename=database, 
    port=port, 
    hostname=host, 
    username=user,
    password=password)

Connecting to the postgreSQL database ...
Database connection created.


## Querying Via the Connection

Now that you're connected to the database, let's take a look at how you can query the data within.

With your cursor object, you can execute queries

In [8]:
cur = conn.cursor()                             # let us create a cursor now 
cur.execute('SELECT * FROM emp')                # execute the select query
result = cur.fetchall()                         # fetch the data and save it to a variable 

In [9]:
result

[(1, 'Sandy', 'Dummy Address'), (2, 'Mandy', 'Dummy Address2')]

In [10]:
cur.description

(Column(name='empid', type_code=23),
 Column(name='empname', type_code=1043),
 Column(name='empaddr', type_code=25))

In [11]:
len(result)

2

It's also possible to combine the previous two cells into one line, like so:

In [ ]:
## Uncomment cell to display contents:
cur.execute("SELECT * FROM actor")

# save it as data
data = cur.fetchall()
data

In [ ]:
data[-1][-1]

### Quick note on formatting syntax:

When working with strings, you may have previously seen a `'string'`, a `"string"`, a `'''string'''`, or a `"""string"""`. While all of these are strings, the triple quotes have the added functionality of being able to use multiple lines within the same string. Sometimes, SQL queries can be much longer than others, in which case it's helpful to use new lines for readability. Here's a short example:

In [ ]:
## Uncomment cell to display contents:

# cur.execute("""SELECT * 
#                FROM emp 
#                LIMIT 5;""").fetchall()

## Wrapping Results Into Pandas DataFrames

Often, a more convenient output will be to turn these results into pandas DataFrames. To do this, you simply wrap the `c.fetchall()` output with a pandas DataFrame constructor:

In [12]:
cur.execute("""SELECT * FROM emp""")                # execute the select query
df = pd.DataFrame(cur.fetchall())                   # fetch the data and convert it to Pandas DataFrame
df.head(10)                   

,0,1,2
0,1,Sandy,Dummy Address
1,2,Mandy,Dummy Address2


Sadly as you can see this is slightly clunky as we do not have the column names. 

We can access the column names by calling `cur.description`, like so:

In [13]:
[x[0] for x in cur.description]

['empid', 'empname', 'empaddr']

In [21]:
# read our config file
from configparser import ConfigParser
config = ConfigParser()
config.read('config.ini')

# get the login credentials 
database = config['dev']['database']
host = config['dev']['host']
port = config['dev']['port']
user = config['dev']['user']
password = config['dev']['password']

# creating a connection to postgresql
conn= ru.create_connection(
    databasename=database, 
    port=port, 
    hostname=host, 
    username=user,
    password=password)

# let us create a cursor now 
cur = conn.cursor()
cur.execute("""SELECT * FROM emp""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head(20)

Connecting to the postgreSQL database ...
Database connection created.


,empid,empname,empaddr
0,1,Sandy,Dummy Address
1,2,Mandy,Dummy Address2


In [22]:
df.dtypes

empid       int64
empname    object
empaddr    object
dtype: object

- Check out the documentation for more info on cursor methods and attributes [here](https://docs.python.org/2/library/sqlite3.html#sqlite3.Cursor).

## The Where Clause

In general, the where clause filters query results by some condition. As you are starting to see, you can also combine multiple conditions.

### Selecting Customers From a Specific City

In [ ]:
cur.execute("""SELECT * FROM emp WHERE empid = 1;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

## Connecting to another table

In [ ]:
database = config['dvdrental']['database']
host = config['dvdrental']['host']
port = config['dvdrental']['port']
user = config['dvdrental']['user']
password = config['dvdrental']['password']

# creating a connection to postgresql
conn= ru.create_connection(
    databasename=database, 
    port=port, 
    hostname=host, 
    username=user,
    password=password)

# let us create a cursor now 
cur = conn.cursor()
cur.execute('SELECT customer_id, active FROM customer where active=1')

customers_df = pd.DataFrame(cur.fetchall())
customers_df.columns = [x[0] for x in cur.description]
customers_df.head()

In [ ]:
customers_df.shape
# (599, xxxx)

In [ ]:
cur.execute('SELECT * FROM rental')
rental_df = pd.DataFrame(cur.fetchall())
rental_df.columns = [x[0] for x in cur.description]
rental_df.head()

In [ ]:
final_df = customers_df.join(rental_df,on='customer_id', how='inner',lsuffix='_caller')[['email','address_id']]
final_df.head()

In [ ]:
customers_df['active'].value_counts()

In [ ]:
active_customers = customers_df[customers_df['active'] ==1]
over_100 = active_customers[active_customers['customer_id']>100]
less_than_200 = over_100[over_100['customer_id']<=200]
less_than_200.shape

In [ ]:
result_df = customers_df[(customers_df['active'] == 1) & (customers_df['customer_id'].between(100,201))][['first_name','last_name','email']]
print(result_df.shape)
result_df.head()

In [ ]:
# Insert INTO
sql_query = """
    INSERT INTO students --(Name, Weight, Height, SmokeOrNot)
    VALUES (%s,%s,%s,%s);"""
cur.execute(sql_query, ('Sara',55,168,'No'))
conn.commit()

In [ ]:
cur.execute("""SELECT * FROM students;""")
new_studnet_df = pd.DataFrame(cur.fetchall())
new_studnet_df.columns = [x[0] for x in cur.description]
new_studnet_df

## Conecting to DVDRental database

In [ ]:
database = config['dvdrental']['database']
host = config['dvdrental']['host']
port = config['dvdrental']['port']
user = config['dvdrental']['user']
password = config['dvdrental']['password']

print('Database: ', database)
print('Host: ', host)
print('Port: ', port)
print('User: ', user)

In [ ]:
# creating a connection to postgresql
conn= ru.create_connection(
    databasename=database, 
    port=port, 
    hostname=host, 
    username=user,
    password=password)

In [ ]:
# Define a schema and table name
schema = 'public'
table_name = 'customer'
sql_query = "SELECT * FROM {}.{}".format(schema, table_name)
values = None 

customers_df = ru.load_data(connection=conn, sql_query=sql_query, values=values)
customers_df.head()

once we have the the data as a dataframe, we need to close the cursor and the connection by calling these two cods `cur.close()`, `conn.close()`

In [ ]:
## close the cursor
cur.close()

## close the connection 
conn.close()

## Summary

In this lesson, you saw how to connect to a SQL database via python and how to subsequently execute queries against that database. Going forward, you'll continue to learn additional keywords for specifying your query parameters!